In [5]:
import os
from dotenv import load_dotenv
from autogen import AssistantAgent, UserProxyAgent

# Load environment variables
load_dotenv()

# Setup API key and specify the model
openai_api_key = os.getenv("OPENAI_API_KEY")
llm_config = {"model": "gpt-4o-mini"}

# Initialize agents with refined Scrum roles, responsibilities, and prompts to estimate productivity rates and calculate effort
product_owner_agent = UserProxyAgent(
    name="Product_Owner",
    system_message=(
        "You are the Product Owner. Represent the customer’s interests by prioritizing and clarifying requirements."
        "During sprint planning, answer questions from the team and ensure that requirements are clear."
        "After each sprint, provide concise feedback on deliverables. Confirm with 'Task Complete: Requirements Clarified.'"
    ),
    llm_config=llm_config
)
#Decides what the team should work on next by setting priorities and making sure the work meets the needs of users or customers.

scrum_master_agent = AssistantAgent(
    name="Scrum_Master",
    system_message=(
        "You are the Scrum Master. Facilitate all Scrum ceremonies, including sprint planning, daily stand-ups, sprint reviews, and retrospectives."
        "Ensure the team stays on track, help resolve blockers, and reinforce Scrum principles. Use 'Task Complete' to confirm the end of each ceremony."
    ),
    llm_config=llm_config
)
#Scrum Master: Helps the team work smoothly by removing any blockers and making sure everyone follows Scrum rules.

project_manager_agent = AssistantAgent(
    name="Project_Manager",
    system_message=(
        "You are the Project Manager. Your role is to oversee sprint progress and coordinate tasks across all team members based on the Product Owner's feedback. "
        "Track and summarize each team member's calculated efforts to keep the sprint on schedule."
    ),
    llm_config=llm_config
)

requirement_engineer_agent = AssistantAgent(
    name="Requirement_Engineer",
    system_message=(
        "You are the Requirement Engineer. Your task is to document and break down requirements for each user story into actionable tasks. "
        "Estimate the number of requirements for each user story and decide on your productivity rate (requirements per day) based on the complexity. "
        "Calculate effort as:\n- Work = total requirements\n- Productivity rate = your estimated requirements/day\n- Effort = Work / Productivity rate."
        "\nProductivity rate = 2 requirements completed every day."
    ),
    llm_config=llm_config
)

system_engineer_agent = AssistantAgent(
    name="System_Engineer",
    system_message=(
        "You are the System Engineer. Design the system architecture based on documented requirements. "
        "Estimate the total number of pages in the design document and decide on your productivity rate (pages per day) based on the complexity. "
        "Calculate effort as:\n- Work = total pages\n- Productivity rate = your estimated pages/day\n- Effort = Work / Productivity rate."
        "\nProductivity rate = 2 pages completed every day."
        
    ),
    llm_config=llm_config
)

software_engineer_agent = AssistantAgent(
    name="Software_Engineer",
    system_message=(
        "You are the Software Engineer. Develop software components for each user story according to the design. "
        "Estimate the number of lines of code (SLOC) needed and decide on your productivity rate (SLOC per day) based on the complexity and see to it that it is realistic the the number of SLOC is not less than 1500. "
        "Calculate effort as:\n- Work = total SLOC\n- Productivity rate = your estimated SLOC/day\n- Effort = Work / Productivity rate."
        "Productivity rate = 50 SLOC completed every day."
    ),
    llm_config=llm_config
)

test_engineer_agent = AssistantAgent(
    name="Test_Engineer",
    system_message=(
        "You are the Test Engineer. Create and execute test cases to validate each feature. "
        "Estimate the number of test cases required and decide on your productivity rate (test cases per day) based on the complexity. "
        "Calculate effort as:\n- Work = total test cases\n- Productivity rate = your estimated test cases/day\n- Effort = Work / Productivity rate."
        "/nProductivity rate = 1 test cases completed every day."
    ),
    llm_config=llm_config
)

documentation_engineer_agent = AssistantAgent(
    name="Documentation_Engineer",
    system_message=(
        "You are the Documentation Engineer. Prepare user documentation for completed features. "
        "Estimate the total number of pages needed for documentation and decide on your productivity rate (pages per day) based on the complexity. "
        "Calculate effort as:\n- Work = total pages\n- Productivity rate = your estimated pages/day\n- Effort = Work / Productivity rate."
        "\nProductivity rate = 1.5 pages completed every day."
    ),
    llm_config=llm_config
)

# Define sprints and user stories
sprints = [
    {"Sprint 1": ["User Registration and Login"]},
    {"Sprint 2": ["Symptom Checker", "Medicine Information"]},
    {"Sprint 3": ["Diet Guide", "Health Tips"]},
    {"Sprint 4": ["User Profile Management", "Feedback Collection", "Logout"]}
]
# Initialize conversation history
conversation_history = ""
initial_message = (
    "I want to create a virtual healthcare assistant chatbot. "
    "Users can log in, check for common symptoms, search for medicines, get diet guides, and receive suggestions for health improvement."
)
conversation_history += f"Customer_Proxy: {initial_message}\n"
# Define the workflow for a Scrum sprint
def run_sprint(sprint_name, sprint_stories):
    global conversation_history
    print(f"--- {sprint_name} ---")
    print(f"User Stories: {', '.join(sprint_stories)}\n")

    # Step 1: Product Owner provides requirements to Scrum Master
    po_response = product_owner_agent.send(
        message={"content": f"Here are the requirements for this sprint: {', '.join(sprint_stories)}."},
        recipient=scrum_master_agent,
        request_reply=True
    )
    print(f"Product_Owner:\n{po_response}\n")
    conversation_history += f"Product_Owner: {po_response}\n"

    # Step 2: Scrum Master asks Requirement Engineer for requirements documentation
    sm_response = scrum_master_agent.send(
        message={"content": f"Please document detailed requirements for the user stories: {', '.join(sprint_stories)}."},
        recipient=requirement_engineer_agent,
        request_reply=True
    )
    print(f"Scrum_Master:\n{sm_response}\n")
    conversation_history += f"Scrum_Master: {sm_response}\n"

    # Requirement Engineer documents requirements and reports back to Scrum Master
    re_response = requirement_engineer_agent.send(
        message={"content": f"Requirements documentation for user stories: {', '.join(sprint_stories)} is complete."},
        recipient=scrum_master_agent,
        request_reply=True
    )
    print(f"Requirement_Engineer:\n{re_response}\n")
    conversation_history += f"Requirement_Engineer: {re_response}\n"

    # Scrum Master informs System Engineer to start system design
    sm_to_se_response = scrum_master_agent.send(
        message={"content": f"Requirements documentation is complete. Please begin system architecture design for user stories: {', '.join(sprint_stories)}."},
        recipient=system_engineer_agent,
        request_reply=True
    )
    print(f"Scrum_Master:\n{sm_to_se_response}\n")
    conversation_history += f"Scrum_Master to System_Engineer: {sm_to_se_response}\n"

    # System Engineer designs architecture and reports back to Scrum Master
    se_response = system_engineer_agent.send(
        message={"content": f"System architecture design for user stories: {', '.join(sprint_stories)} is complete."},
        recipient=scrum_master_agent,
        request_reply=True
    )
    print(f"System_Engineer:\n{se_response}\n")
    conversation_history += f"System_Engineer: {se_response}\n"

    # Scrum Master informs Software Engineer to start development
    sm_to_sw_response = scrum_master_agent.send(
        message={"content": f"System architecture design is complete. Please proceed with detailed software development for user stories: {', '.join(sprint_stories)}."},
        recipient=software_engineer_agent,
        request_reply=True
    )
    print(f"Scrum_Master:\n{sm_to_sw_response}\n")
    conversation_history += f"Scrum_Master to Software_Engineer: {sm_to_sw_response}\n"

    # Software Engineer develops the software and reports back to Scrum Master
    sw_response = software_engineer_agent.send(
        message={"content": f"Software development for user stories: {', '.join(sprint_stories)} is complete."},
        recipient=scrum_master_agent,
        request_reply=True
    )
    print(f"Software_Engineer:\n{sw_response}\n")
    conversation_history += f"Software_Engineer: {sw_response}\n"

    # Scrum Master informs Test Engineer to start testing
    sm_to_te_response = scrum_master_agent.send(
        message={"content": f"Software development is complete. Please proceed with testing for user stories: {', '.join(sprint_stories)}."},
        recipient=test_engineer_agent,
        request_reply=True
    )
    print(f"Scrum_Master:\n{sm_to_te_response}\n")
    conversation_history += f"Scrum_Master to Test_Engineer: {sm_to_te_response}\n"

    # Test Engineer completes testing and reports back to Scrum Master
    te_response = test_engineer_agent.send(
        message={"content": f"Testing for user stories: {', '.join(sprint_stories)} is complete."},
        recipient=scrum_master_agent,
        request_reply=True
    )
    print(f"Test_Engineer:\n{te_response}\n")
    conversation_history += f"Test_Engineer: {te_response}\n"

    # Scrum Master informs Documentation Engineer to start documentation
    sm_to_doc_response = scrum_master_agent.send(
        message={"content": f"Testing is complete. Please proceed with documentation for user stories: {', '.join(sprint_stories)}."},
        recipient=documentation_engineer_agent,
        request_reply=True
    )
    print(f"Scrum_Master:\n{sm_to_doc_response}\n")
    conversation_history += f"Scrum_Master to Documentation_Engineer: {sm_to_doc_response}\n"

    # Documentation Engineer completes documentation and reports back to Scrum Master
    doc_response = documentation_engineer_agent.send(
        message={"content": f"Documentation for user stories: {', '.join(sprint_stories)} is complete. Here is the summary."},
        recipient=scrum_master_agent,
        request_reply=True
    )
    print(f"Documentation_Engineer:\n{doc_response}\n")
    conversation_history += f"Documentation_Engineer: {doc_response}\n"

# Run each sprint in sequence
for sprint in sprints:
    sprint_name = list(sprint.keys())[0]
    sprint_stories = sprint[sprint_name]
    
    # Start the sprint
    run_sprint(sprint_name, sprint_stories)

    # Optionally, human review or feedback after each sprint
    feedback = input(f"Would you like to review {sprint_name} or continue? (Type 'review' or 'continue'): ").lower()
    
    if feedback == 'review':
        review_response = customer_proxy_agent.send(
            message={"content": f"Please review the sprint outcomes for {sprint_name} and provide feedback."},
            recipient=scrum_master_agent,
            request_reply=True
        )
        print(f"Customer_Proxy Feedback:\n{review_response}\n")
        conversation_history += f"Customer_Proxy: {review_response}\n"

    # Option to exit after feedback
    continue_sprints = input("Would you like to continue to the next sprint or stop? (Type 'continue' or 'stop'): ").lower()
    if continue_sprints == 'stop':
        print("Exiting the process.")
        break

print("All sprints completed or exited.")


[autogen.oai.client: 11-01 16:27:58] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.
[autogen.oai.client: 11-01 16:27:59] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.
[autogen.oai.client: 11-01 16:28:00] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.
[autogen.oai.client: 11-01 16:28:01] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.
[autogen.oai.client: 11-01 16:28:02] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.
[autogen.oai.client: 11-01 16:28:03] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.
[autogen.oai.client: 11-01 16:28:04] {184} WARNING - The API key specified is not a valid OpenAI format; i

Would you like to review Sprint 1 or continue? (Type 'review' or 'continue'):  
Would you like to continue to the next sprint or stop? (Type 'continue' or 'stop'):  


--- Sprint 2 ---
User Stories: Symptom Checker, Medicine Information

Product_Owner (to Scrum_Master):

Here are the requirements for this sprint: Symptom Checker, Medicine Information.

--------------------------------------------------------------------------------
Scrum_Master (to Product_Owner):

Thank you, Product Owner. Since we have new requirements, let's proceed with the Sprint Planning to ensure we capture all the necessary details and tasks for these features.

**Sprint Planning:**

Team, let's delve into the new requirements and break them down into actionable tasks.

1. **Symptom Checker:**
   - Requirements Gathering: Identify symptoms to be included and their mapping to possible conditions.
   - Design: Create the symptom checker interface and workflow.
   - Develop Front-End: UI for symptom entry and results display.
   - Implement Backend Logic: Develop algorithms or integrate with existing databases for symptom analysis.
   - Testing: Write test cases covering differe

Would you like to review Sprint 2 or continue? (Type 'review' or 'continue'):  
Would you like to continue to the next sprint or stop? (Type 'continue' or 'stop'):  


--- Sprint 3 ---
User Stories: Diet Guide, Health Tips

Product_Owner (to Scrum_Master):

Here are the requirements for this sprint: Diet Guide, Health Tips.

--------------------------------------------------------------------------------
Scrum_Master (to Product_Owner):

It seems like we're receiving multiple sets of requirements for sprint planning. Could you please confirm which set of requirements you'd like us to focus on for this sprint? Here are the options based on your previous inputs:

1. **User Registration and Login**
2. **Symptom Checker and Medicine Information**
3. **Diet Guide and Health Tips**

Once you've confirmed, we can proceed with the Sprint Planning session tailored to your chosen set of requirements.

--------------------------------------------------------------------------------
Product_Owner:
None

Scrum_Master (to Requirement_Engineer):

Please document detailed requirements for the user stories: Diet Guide, Health Tips.

----------------------------------

Would you like to review Sprint 3 or continue? (Type 'review' or 'continue'):  
Would you like to continue to the next sprint or stop? (Type 'continue' or 'stop'):  


--- Sprint 4 ---
User Stories: User Profile Management, Feedback Collection, Logout

Product_Owner (to Scrum_Master):

Here are the requirements for this sprint: User Profile Management, Feedback Collection, Logout.

--------------------------------------------------------------------------------
Scrum_Master (to Product_Owner):

Thank you for providing the requirements. Let's organize a Sprint Planning meeting to focus on these features and ensure we have clarity on the tasks involved.

**Sprint Planning:**

Team, we have three new features to work on: User Profile Management, Feedback Collection, and Logout. Let's break these down.

1. **User Profile Management:**
   - Requirements Gathering: What information can users manage in their profiles?
   - Design: Mockups for profile creation and editing screens.
   - Develop Front-End: Build UI components for profile management.
   - Implement Backend: APIs for fetching and updating user profiles.
   - Testing: Write scenarios for updating

Would you like to review Sprint 4 or continue? (Type 'review' or 'continue'):  
Would you like to continue to the next sprint or stop? (Type 'continue' or 'stop'):  


All sprints completed or exited.
